In [1]:
import pandas as pd
import sqlite3
import os
import numpy as np
from datetime import datetime, timedelta,date
from dotenv import dotenv_values
from binance.client import Client

 # SELECT symbol_pair,count(*) as  no_tran from hist_trade  
 #        where datetimeUTC >'2023-01-01' GROUP by symbol_pair

 # SELECT * from hist_trade  
 #        where datetimeUTC >'2023-01-01' order by  datetimeUTC desc

In [2]:
# create database  DB_BN_MyTrade.db with 2 tables as schema below. 
# CREATE TABLE "hist_trade" (
# 	"datetimeUTC"	TEXT,
# 	"symbol_pair"	TEXT,
# 	"symbol"	TEXT,
# 	"pair"	TEXT,
# 	"type"	TEXT,
# 	"price"	REAL,
# 	"qty"	REAL,
# 	"total"	REAL,
# 	"commission"	REAL,
# 	"commissionAsset"	TEXT
# );


# CREATE TABLE "symbol_info" (
# 	"symbol"	TEXT,
# 	"pair"	TEXT,
# 	"isActive"	INTEGER
# );


#sample data "hist_trade"
# 2022-08-13 01:18:00,MATICBUSD,MATIC,BUSD,SELL,0.94,22.0,20.68,0.0,BNB

# sample data in   "symbol_info"
# symbol,pair,active
# ETH,BUSD,1

In [3]:
path_config=r'D:\AB_DB\Script_ImportData'
config_values= dotenv_values(dotenv_path=f'{path_config}\\.env')
api_key = config_values['bn_key']
api_secret =config_values['bn_secrete']
client=Client(api_key,api_secret)

In [4]:
today=date.today()
today=datetime(today.year,today.month,today.day)
print(today)

2023-10-31 00:00:00


In [5]:
data_base_file=f'{path_config}\\Database\\DB_BN_MyTrade.db'
print("Connect SQLite")
conn = sqlite3.connect(os.path.abspath(data_base_file))
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

Connect SQLite


In [6]:
sql_symbol="select * from symbol_info where isActive=1"
dfSymbols=pd.read_sql_query(sql_symbol, conn)
dfSymbols['symbol_pair']=dfSymbols.apply( lambda x: f"{x.symbol}{x.pair}",axis=1)
dfSymbols = dfSymbols.drop_duplicates(keep='first')
dfSymbols=dfSymbols.set_index('symbol_pair')
dfSymbols

,symbol,pair,isActive
symbol_pair,,,
ETHBUSD,ETH,BUSD,1
BTCBUSD,BTC,BUSD,1
BNBBUSD,BNB,BUSD,1
MATICBUSD,MATIC,BUSD,1


# Get data from Binanac and transform

In [7]:
cols=['datetimeUTC','symbol_pair','symbol','pair','type','price','qty','total','commission','commissionAsset']

In [8]:
def split_main_pair_symbol(symbol_pair,main):
    pair=''
    main_idx=symbol_pair.rfind(main)
    if main_idx==0:
     pair= symbol_pair.replace(main, '')
    return main,pair

In [9]:
def transform_trans_to_df(tradesList,symbol_main):
    dfTrade=pd.DataFrame(data=tradesList)
    
    dfTrade["datetimeUTC"]=pd.to_datetime(dfTrade["time"],unit="ms")
    dfTrade["datetimeUTC"]=dfTrade['datetimeUTC'].dt.strftime("%Y-%m-%d %H:%M:%S")
    dfTrade["datetimeUTC"]=pd.to_datetime(dfTrade["datetimeUTC"],format="%Y-%m-%d %H:%M:%S")
    
    dfTrade['type']= dfTrade['isBuyer'].apply( lambda x : 'BUY' if x==True else 'SELL')
    # dfTrade['total']=dfTrade['price']*dfTrade['qty']
    dfTrade=dfTrade.rename(columns={'symbol':'symbol_pair','quoteQty':'total'})
    
    main,pair=split_main_pair_symbol(symbol_pair,symbol_main)
    dfTrade['symbol']=main
    dfTrade['pair']=pair
    
    cols_number=['price','qty','total','commission']
    dfTrade[cols_number]= dfTrade[cols_number].astype("float")
    
    
    dfTrade=dfTrade[cols]
    return dfTrade

In [10]:
sr_symbol_pair=dfSymbols['symbol']
print(sr_symbol_pair)

symbol_pair
ETHBUSD        ETH
BTCBUSD        BTC
BNBBUSD        BNB
MATICBUSD    MATIC
Name: symbol, dtype: object


In [11]:
dfAllTran=pd.DataFrame(columns=cols)

In [12]:
for  item in sr_symbol_pair.iteritems():
    main_pair=item[0]
    main=item[1]
    sql_last_trans=f""" SELECT * from hist_trade  
        where symbol_pair='{main_pair}' order by datetimeUTC desc limit 1 """

    dfLastTran=pd.read_sql_query(sql_last_trans, conn)
    if dfLastTran.empty==False:
        print(f'====================Last Transaction of {main}==============================')
        print(dfLastTran[['datetimeUTC','symbol_pair','type','price','qty','total']])
        srLastTran=dfLastTran.iloc[0,:]
        symbol_pair=srLastTran['symbol_pair']
        dtLastStr=srLastTran['datetimeUTC']
    
        dtLast=datetime.strptime(dtLastStr, '%Y-%m-%d %H:%M:%S')
        startDate=datetime.strptime(dtLastStr, "%Y-%m-%d %H:%M:%S").date()
        startDateTime=datetime(startDate.year, startDate.month, startDate.day)
        
        
        # while startDateTime<=today:
        try:    
            statrTimestampMs= (int(round(startDateTime.timestamp())))* 1000
            trades_of_x = client.get_my_trades(symbol=symbol_pair,startTime=statrTimestampMs)
            if len(trades_of_x)>0:
                df_daily_trades=transform_trans_to_df(trades_of_x,main)
                df_daily_trades=df_daily_trades.query('datetimeUTC>@dtLast')
                if  df_daily_trades.empty ==False:
                 print(df_daily_trades[['datetimeUTC','symbol_pair','type','price','qty','total']])
                 dfAllTran=pd.concat([dfAllTran,df_daily_trades])
        except BinanceAPIException as e:
              print(e)
            
        # startDateTime=startDateTime+timedelta(days=1)    

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_18208\456605943.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for  item in sr_symbol_pair.iteritems():


====================Last Transaction of ETH==============================
           datetimeUTC symbol_pair  type   price   qty   total
0  2022-12-16 09:00:22     ETHBUSD  SELL  1239.6  0.02  24.792
            datetimeUTC symbol_pair  type    price     qty      total
1   2023-02-14 12:34:07     ETHBUSD   BUY  1511.57  0.0200  30.231400
2   2023-02-15 18:45:39     ETHBUSD   BUY  1594.17  0.0127  20.245959
3   2023-02-15 18:45:39     ETHBUSD   BUY  1594.18  0.0123  19.608414
4   2023-02-16 22:31:36     ETHBUSD   BUY  1660.00  0.0180  29.880000
5   2023-02-21 21:22:08     ETHBUSD  SELL  1649.40  0.0148  24.411120
..                  ...         ...   ...      ...     ...        ...
106 2023-08-01 12:42:19     ETHBUSD   BUY  1833.84  0.0150  27.507600
107 2023-08-01 12:43:48     ETHBUSD   BUY  1832.97  0.0100  18.329700
108 2023-08-01 15:30:32     ETHBUSD  SELL  1814.62  0.0125  22.682750
109 2023-08-01 23:30:45     ETHBUSD  SELL  1870.00  0.0125  23.375000
110 2023-08-09 15:14:16     ET

# Add to SQLite3

In [13]:
def insertTradeTransaction(recordList):
    try:
        sqliteConnection = sqlite3.connect(os.path.abspath(data_base_file))
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        sqlite_insert_query = """
        INSERT INTO hist_trade
         VALUES (?,?,?,?,?,?,?,?,?,?);
         """

        cursor.executemany(sqlite_insert_query, recordList)
        sqliteConnection.commit()
        print("Total ", cursor.rowcount, " Add transaction successfully")
        sqliteConnection.commit()
        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert multiple records to table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            print("The SQLite connection is closed")
            


In [14]:
# from google.cloud import bigquery
# table_id = "pongthorn.FinDW.bn_hist_trade"
# def insertTradeToBQ(df_trasns):
#     client = bigquery.Client()
#     job_config = bigquery.LoadJobConfig(

#         schema=[
#             bigquery.SchemaField("datetimeUTC", bigquery.enums.SqlTypeNames.DATETIME),
#             bigquery.SchemaField("symbol_pair", bigquery.enums.SqlTypeNames.STRING),
#             bigquery.SchemaField("symbol", bigquery.enums.SqlTypeNames.STRING),
#             bigquery.SchemaField("pair", bigquery.enums.SqlTypeNames.STRING),
#             bigquery.SchemaField("type", bigquery.enums.SqlTypeNames.STRING),
#             bigquery.SchemaField("price", bigquery.enums.SqlTypeNames.FLOAT64),
#             bigquery.SchemaField("qty", bigquery.enums.SqlTypeNames.FLOAT64),
#             bigquery.SchemaField("total", bigquery.enums.SqlTypeNames.FLOAT64),
#             bigquery.SchemaField("commission", bigquery.enums.SqlTypeNames.FLOAT64),
#             bigquery.SchemaField("commissionAsset", bigquery.enums.SqlTypeNames.STRING),

#         ],
#         write_disposition="WRITE_APPEND",
#     )

#     job = client.load_table_from_dataframe(
#         df_trasns, table_id, job_config=job_config
#     )  # Make an API request.
#     job.result()  # Wait for the job to complete.
#     print("Total ", len(df_trasns), " Add transaction to google bigquery successfully")

In [15]:
if dfAllTran.empty==False:
    dfAllTran=dfAllTran.reset_index(drop=True)
    dfAllTran = dfAllTran.drop_duplicates(keep='last')
    dfAllTran["datetimeUTC"]=dfAllTran['datetimeUTC'].dt.strftime("%Y-%m-%d %H:%M:%S")  # for add to sqlite
    print(dfAllTran.info())
    print(dfAllTran[['datetimeUTC','symbol_pair','type','price','qty','total']])
    
    recordsToInsert=list(dfAllTran.to_records(index=False))
    insertTradeTransaction(recordsToInsert)
    
    dfAllTran['datetimeUTC']=pd.to_datetime(dfAllTran["datetimeUTC"],format="%Y-%m-%d %H:%M:%S")  # for add to bigquery
    # insertTradeToBQ(dfAllTran)
else:
    print("no transsaction update")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   datetimeUTC      1 non-null      object 
 1   symbol_pair      1 non-null      object 
 2   symbol           1 non-null      object 
 3   pair             1 non-null      object 
 4   type             1 non-null      object 
 5   price            1 non-null      float64
 6   qty              1 non-null      float64
 7   total            1 non-null      float64
 8   commission       1 non-null      float64
 9   commissionAsset  1 non-null      object 
dtypes: float64(4), object(6)
memory usage: 88.0+ bytes
None
           datetimeUTC symbol_pair type    price   qty    total
0  2023-07-03 15:35:07     ETHBUSD  BUY  1976.12  0.01  19.7612
Connected to SQLite
Total  1  Add transaction successfully
The SQLite connection is closed
